In [ ]:
import time
from random import randint

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython import display
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd



In [ ]:
class SimpleConvAutoEncoder(tf.keras.Model):
    def __init__(self):
        super(SimpleConvAutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential(
            [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, keyrnel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation=tf.nn.relu),
            tf.keras.layers.Dense(10),
            ]
        )
            
        

        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(10,)),
                tf.keras.layers.Dense(512, activation=tf.nn.relu),
                tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(7,7,32)),
                tf.keras.layers.Conv2DTranspose(
                    filters=64,
                    kernel_size=3,
                    strides=(2,2),
                    padding = 'SAME',
                    activation='relu'
                ),
                tf.keras.layers.Conv2DTranspose(
                    filters=32,
                    kernel_size=3,
                    strides=(2, 2),
                    padding='SAME',
                    activation='relu'
                ),
                tf.keras.layers.Conv2DTranspose(
                    filters=1,
                    kernel_size=3,
                    strides=(1, 1),
                    padding='SAME',
                ),
            ]
        )
    def Encode(self, x):
        return self.encoder(x)

    def Decode(self, x):
        logits = self.decoder(x)
        return tf.sigmoid(logits)
    
    def custom_save_model(self):
      print('saving model..')
      self.encoder.save_weights('drive/My Drive/Colab Notebooks/encoder_model.h5')
      self.decoder.save_weights('drive/My Drive/Colab Notebooks/decoder_model.h5')

    def custom_load_model(self):
      print('loading model..')
      self.encoder.load_weights('drive/My Drive/Colab Notebooks/encoder_model.h5')
      self.decoder.load_weights('drive/My Drive/Colab Notebooks/decoder_model.h5')

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model.Decode(test_input)
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0], cmap='gray')
      plt.axis('off')

  # tight_layout minimizes the overlap between 2 sub-plots
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()



In [ ]:
def train_and_save_results(model, train_set, test_set):
    epochs = 100
    optimizer =tf.keras.optimizers.Adam()
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        for train_x in train_set:
            print(np.sum(compute_loss(model, train_x)))
            compute_apply_gradients(model, train_x, optimizer)
        end_time = time.time()
        loss = np.inf
        if epoch:
            for test_x in test_set:
                loss = np.sum(compute_loss(model, test_x))
            display.clear_output(wait=False)
            print('Epoch: {}, Test set ELBO: {}, '
                  'time elapse for current epoch {}'.format(epoch,
                                                            loss,
                                                            end_time - start_time))
            
            model.custom_save_model()
            if not (epoch % 10) or epoch == 1: 
                generate_and_save_images(model, epoch, random_vector_for_generation)
                train_dataset, test_dataset, train_labels = load_data(1)
                create_2d_plot(train_dataset, train_labels)
                # plot_pca(df)


In [ ]:
# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.



In [ ]:
@tf.function
def compute_loss(model : SimpleConvAutoEncoder,x_real):
    z = model.Encode(x_real)
    logits = model.decoder(z)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=x_real)
    return tf.reduce_mean(loss)

In [ ]:
@tf.function
def compute_apply_gradients(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:

def load_data(batch_size):
    TRAIN_BUF = 60000
    BATCH_SIZE = batch_size
    TEST_BUF = 10000
    (train_images, train_labels), (test_images, _) = tf.keras.datasets.mnist.load_data()
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

    # Normalizing the images to the range of [0., 1.]
    train_images /= 255.
    test_images /= 255.
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images).batch(BATCH_SIZE)
    train_labels = tf.data.Dataset.from_tensor_slices(train_labels).batch(BATCH_SIZE)    
    test_dataset = tf.data.Dataset.from_tensor_slices(test_images).batch(BATCH_SIZE)
    
    #train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(TRAIN_BUF).batch(BATCH_SIZE)
    #train_labels = tf.data.Dataset.from_tensor_slices(train_images).shuffle(TRAIN_BUF).batch(BATCH_SIZE)    
    #test_dataset = tf.data.Dataset.from_tensor_slices(test_images).shuffle(TEST_BUF).batch(BATCH_SIZE)
    return train_dataset ,test_dataset , train_labels

In [ ]:
def plot_pca(df):
  fig = plt.figure(figsize = (8,8))
  ax = fig.add_subplot(1,1,1) 
  ax.set_xlabel('Principal Component 1', fontsize = 15)
  ax.set_ylabel('Principal Component 2', fontsize = 15)
  ax.set_title('2 component PCA', fontsize = 20)
  ax.scatter(df['pc1'], df['pc2'])
  # targets = set(df['target'].values)
  # colors = {'b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 0.2 , 0.8}
  # for target, color in zip(targets,colors):
      # indicesToKeep = df['target'] == target
      # ax.scatter(df.loc[indicesToKeep, 'principal component 1']
                # , df.loc[indicesToKeep, 'principal component 2']
                # , c = color
                # , s = 50)
  # ax.legend(targets)
  ax.grid()
  plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def create_2d_plot(train_dataset, train_labels):
  model = SimpleConvAutoEncoder()
  model.custom_load_model()
  print('creating encoded set')
  sample = np.random.sample((500,10))
  train_dataset = train_dataset.from_tensor_slices(sample)
  train_labels = train_labels.from_tensor_slices(sample)
  zs = [model.Encode(z)[0] for z in train_dataset]  
  arr = np.array(zs)
  # print(zs[0])
  from sklearn.preprocessing import StandardScaler
  # X = StandardScaler().fit_transform(arr)
  from sklearn.decomposition import PCA
  from sklearn.feature_extraction.text import TfidfVectorizer
  from yellowbrick.text import TSNEVisualizer

  print('creating and transforming tsne')
  labels = ['0 num', '1 num', '2 num ', '3 num', '4 num ', '5 num ', '6 num ', '7 num', '8 num','9 num']
  tsne = TSNEVisualizer(decompose_by=5, labels= labels)
  train_labels = [l.numpy()[0] for l in train_labels]
  tsne.fit(zs, train_labels)
  tsne.finalize()
  # pca = PCA(n_components=2)
  # pcs = pca.fit_transform(arr)

  print('creating df')
  # principalDf = pd.DataFrame(data = pcs
            #  , columns = ['pc1', 'pc2'])
  # print('concating df')
  # principalDf['target'] = train_labels
  # return principalDf

  
  

In [ ]:

tf.enable_eager_execution()
random_vector_for_generation = tf.random.normal(
    shape=[16, 10])
print(tf.version)
model = SimpleConvAutoEncoder()
model.custom_load_model()
train_dataset, test_dataset, train_labels = load_data(100)
train_and_save_results(model, train_dataset, test_dataset)






# New Section